# Geospatial prep

Transform the "stage 1" prepped dataset to a "stage 2".

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import time
from collections import defaultdict, Counter
import json
from urllib.request import Request, urlopen
import pickle

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import shape, Point
import shapefile
from geopy import distance
import plotly.figure_factory as ff
import plotly.io as pio
import plotly as plt
pio.renderers.default = "jupyterlab"
import plotly.express as px
import plotly.graph_objects as go

In [4]:
from tqdm import tqdm

In [5]:
from utils import *

In [ ]:
df = load_csv_export_dataset("export_dataframe_stage1.csv")

In [36]:
locs = df[['SiteName', 'SiteLocation']]
len(locs)

In [10]:
locs.head()

,SiteName,SiteLocation
0,West Fork of Little River,34.4103 -83.8462
1,West Fork of Little River,34.4103 -83.8462
2,Apalachee River,33.8406 -83.5584
3,Apalachee River,33.8406 -83.5584
4,Apalachee River,33.8406 -83.5584


# NULLs and test sites

In [47]:
df[df['SiteLocation'].isna()]['SiteName'].value_counts()

Test Site    154
Name: SiteName, dtype: int64

This shows that all empty locations are associated with "Test Site", and strongly suggests the whole records are not real data points.

So let's remove 'NULL' locations in the CSV (recorded as np.nan in the dataframe parsing) locations from the dataset.

We might also provide a means to remove the full records from the dataset and update the data for other users.

In [291]:
df = df[~df['SiteLocation'].isna()]

In [38]:
#locs = locs[locs['SiteLocation'] != 'NULL']
locs = locs[~locs['SiteLocation'].isna()]

Notice how these coordinates are stored in an awkward string format. Let's break these out to pairs of floats, which will enable us to do more with GIS utilities.

In [35]:
loc_pairs, geo_locs, gdf = get_loc_objects_from_series(locs['SiteLocation'])

# Distribution of locations

Make sure to unzip the file in the geodata folder first.

In [8]:
# watershed boundaries
WBD_gj = shapefile.Reader("../geodata/hydrologic_units_WBDHU12_ga_3975106_02/hydrologic_units\wbdhu12_a_ga.shp").__geo_interface__

Demonstrate that the max inter-site distance is < 450 miles

In [84]:
GA_map = get_state_map()

In [21]:
GA_map.head()

,FIPS,STNAME,County,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,NHWA_MALE,NHWA_FEMALE,NHWhite_Alone,Not_NHWhite_Alone,MinorityMinority,MinorityPCT,Black,BlackPCT,Hispanic,HispanicPCT
2807,13001,Georgia,Appling,18368,9294,9074,7197,7139,6284,6520,12804,5564,No,30.29%,3607,20%,1728,9.41%
2808,13003,Georgia,Atkinson,8284,4203,4081,3291,3143,2317,2371,4688,3596,No,43.41%,1459,18%,2050,24.75%
2809,13005,Georgia,Bacon,11198,5562,5636,4558,4592,4094,4273,8367,2831,No,25.28%,1790,16%,890,7.95%
2810,13007,Georgia,Baker,3366,1606,1760,858,876,796,823,1619,1747,Yes,51.90%,1551,46%,152,4.52%
2811,13009,Georgia,Baldwin,46367,23365,23002,13011,12750,12638,12391,25029,21338,No,46.02%,19150,41%,977,2.11%


In [85]:
fips = GA_map['FIPS'].tolist()

In [226]:
len(WBD_gj['features'])

1860

In [229]:
wbd_names = set([wbd['properties']['name'] for wbd in WBD_gj['features']])

In [232]:
# some overlap in site names to watershed boundaries
len([sn for sn in df['SiteName'].unique() if sn in wbd_names])

137

### Plot distribution of unique record locations using mapbox

In [ ]:
fig = px.choropleth_mapbox(pd.DataFrame(data={'name': [wbd['properties']['name'] for wbd in WBD_gj['features']]}),
                           geojson=WBD_gj,
                           locations="name",
                           featureidkey="properties.name",
                           center={"lat": 32.8407, "lon": -83.6324}, # macon
                           mapbox_style="stamen-terrain", #"carto-positron",
                           zoom=6,
                           opacity=0.3,
                      )

fig.add_scattermapbox(
    lat = gdf.geometry.x,
    lon = gdf.geometry.y,
    mode = 'markers',
    hovertext = [d['County']['name'] for d in loc_lookup],
    marker_size=4,
    marker_color='rgb(235, 0, 100)'
)

fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
fig.show()

### Alternate plot of distribution of unique record locations

In [ ]:
fig = go.Figure(go.Scattergeo())
fig.layout.template = None
fig.add_scattergeo(
    lat = gdf.geometry.x,
    lon = gdf.geometry.y,
    hoverinfo='none',
    #mode = 'markers+text',
    #text = texts,
    marker_size = 2,
    marker_color = 'rgb(235, 0, 100)'
)
fig.update_geos(
    visible=False, resolution=110, scope="usa",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Black"
)
fig.show()

## Check on recorded locations from outside the USA!

In [186]:
non_usa_count = []
for i, d in enumerate(loc_lookup):
    if d['State']['FIPS'] is None:
        non_usa_count.append(i)
len(non_usa_count)

3

In [279]:
non_usa_count

[0, 1, 379]

Let's find them in the dataframe

In [336]:
df[df['long'] < -96]

,group_rid,GroupName,site_rid,state,county,long,lat,SiteName,SiteLocation,event_rid,...,hold_end_datetime,min_temp,max_temp,three_M_plate,ecoli_idexx,fecal_coliform,ecoli_other,ecoli_other_unit,comments,warnings
35029,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35410,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35411,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36391,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,57519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37542,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,59997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [337]:
df[df['group_rid'] == 2359]

,group_rid,GroupName,site_rid,state,county,long,lat,SiteName,SiteLocation,event_rid,...,hold_end_datetime,min_temp,max_temp,three_M_plate,ecoli_idexx,fecal_coliform,ecoli_other,ecoli_other_unit,comments,warnings
35029,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35410,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35411,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36391,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,57519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37542,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,59997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [338]:
df[df['GroupName'] == "Stream'n at Pin Point"]

,group_rid,GroupName,site_rid,state,county,long,lat,SiteName,SiteLocation,event_rid,...,hold_end_datetime,min_temp,max_temp,three_M_plate,ecoli_idexx,fecal_coliform,ecoli_other,ecoli_other_unit,comments,warnings
35029,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35410,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35411,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36391,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,57519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37542,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,59997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
df[df['SiteName'] == "Gullah Gullah Creek"]

,group_rid,GroupName,site_rid,state,county,long,lat,SiteName,SiteLocation,event_rid,...,hold_end_datetime,min_temp,max_temp,three_M_plate,ecoli_idexx,fecal_coliform,ecoli_other,ecoli_other_unit,comments,warnings
35029,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35410,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35411,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,56440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36391,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,57519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37542,2359,Stream'n at Pin Point,4557,None,None,-96.0333,18.97,Gullah Gullah Creek,18.97 -96.0333,59997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Results:
 - observation:
     - there are 5 SiteLocations indicating to be outside of the USA, all at the same spot in Mexico
     - The records are labelled with GroupName "Stream'n at Pin Point" with group_rid 2359 and SiteName "Gullah Gullah Creek"
     - Those 5 records are the only ones recorded by that group and at that SiteName
     - The name "Gullah Gullah Creek" has no hits on Google, but "Gullah" is associated with an area near Charleston, SC.
 - source: plotting the (lat, lon) on a map
 - conclusion: we should remove these records from the dataset as not being close to GA

We'll remove these anyway by virtue of the WBD screening that comes later!

### Sidebar: How complete are these out-of-US records anyway?

In [377]:
gullah_df = df[df['SiteName'] == "Gullah Gullah Creek"]

In [389]:
gullah_df[field_groups["core"]]

,event_date,volunteer_time,createdby,createddate,data_entry,participants
35029,2016-12-09 02:30:00,NaN,EPDMIG,3/10/17 11:48 PM,36434.0,1
35410,2017-01-13 13:00:00,NaN,EPDMIG,3/10/17 11:48 PM,36434.0,2
35411,2017-01-20 13:30:00,NaN,EPDMIG,3/10/17 11:48 PM,36434.0,2
36391,2017-04-11 11:00:00,65.0,lynn.neagley12@gmail.com,4/11/17 9:22 PM,36434.0,1
37542,2017-08-17 15:15:00,65.0,lynn.neagley12@gmail.com,8/20/17 9:53 PM,36434.0,1


In [398]:
completion_by_fieldgroup(gullah_df)

{'core': 0.9,
 'basic': 0.3473684210526316,
 'chem': 0.3181818181818182,
 'bact': 0.06666666666666667,
 'other': 0.0}

In [399]:
completion_by_fieldgroup(df)

{'core': 0.9295343041208834,
 'basic': 0.31316272090009184,
 'chem': 0.20733876058577846,
 'bact': 0.2026678370456291,
 'other': 0.000986387626825265}

## Show example of location being way outside of the USA

In [ ]:
fig = go.Figure(go.Scattergeo())
fig.layout.template = None
fig.add_scattergeo(
    lat = [gdf.geometry.x[0]],
    lon = [gdf.geometry.y[0]],
    hoverinfo='none',
    #mode = 'markers+text',
    #text = texts,
    marker_size = 10, #2,
    marker_color = 'rgb(235, 0, 100)'
)
fig.update_geos(
    visible=False, resolution=110, #scope="usa",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Black"
)
fig.show()

## Other non-GA locations of records

We'll compare locations to the borders of GA state using state GIS shape data and an algorithm based on drawing a line from a given measurement location to the closest state boundary line segment. The source of this approach is from https://www.reddit.com/r/dataisbeautiful/comments/6ulj3v/the_longest_straight_line_that_can_be_drawn_in/

In [341]:
ga_locs = []
states = []
non_ga_idxs = defaultdict(list)
for i, d in enumerate(loc_lookup):
    if i in non_usa_count:
        continue
    else:
        state_name = d['State']['name']
        states.append(state_name)
        if state_name == "Georgia":
            ga_locs.append(tuple(loc_pairs[i]))
        else:
            non_ga_idxs[state_name].append(i)
Counter(states).most_common()

[('Georgia', 2570),
 ('Florida', 142),
 ('South Carolina', 105),
 ('North Carolina', 60),
 ('Tennessee', 25),
 ('Alabama', 12)]

In [372]:
SC_df = df[df['state'] == 'South Carolina']

In [373]:
SC_df.head()

,group_rid,GroupName,site_rid,state,county,long,lat,SiteName,SiteLocation,event_rid,...,hold_end_datetime,min_temp,max_temp,three_M_plate,ecoli_idexx,fecal_coliform,ecoli_other,ecoli_other_unit,comments,warnings
5785,1087,AAAS Stream Stompers,953,South Carolina,Aiken,-81.8534,33.3345,Hollow Creek 1,33.3345 -81.8534,22175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5788,1087,AAAS Stream Stompers,953,South Carolina,Aiken,-81.8534,33.3345,Hollow Creek 1,33.3345 -81.8534,22179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5789,1087,AAAS Stream Stompers,954,South Carolina,Aiken,-81.8222,33.3434,Hollow Creek 2,33.3434 -81.8222,22181,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5790,1087,AAAS Stream Stompers,954,South Carolina,Aiken,-81.8222,33.3434,Hollow Creek 2,33.3434 -81.8222,22182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5794,1087,AAAS Stream Stompers,953,South Carolina,Aiken,-81.8534,33.3345,Hollow Creek 1,33.3345 -81.8534,22186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
_, _, gdf_sc = get_loc_objects_from_series(SC_df['SiteLocation'])

### How far are these non-GA locations from the GA border?

In [183]:
from spatial_utils import *

In [189]:
hull = ConvexHull(ga_locs)

In [190]:
ga_hull_points = np.array(ga_locs)[hull.vertices]

In [312]:
dist_to_ga = {}
for state, idxs in tqdm(non_ga_idxs.items()):
    dist_to_ga[state] = {}
    for idx in idxs:
        p = loc_pairs[idx]
        close_pts = []
        for i in range(len(ga_hull_points)-1):
            close_pts.append(dist_to_line(ga_hull_points[i][0],
                                            ga_hull_points[i][1],
                                            ga_hull_points[i+1][0],
                                            ga_hull_points[i+1][1],
                                            p[0], p[1]))
        sq_dists = [sq_dist(cp[0], cp[1], p[0], p[1]) for cp in close_pts]
        ci = np.argmin(sq_dists)
        dist_to_ga[state][idx] = distance.distance(ga_hull_points[ci], p).miles

100%|██████████| 5/5 [00:00<00:00, 43.98it/s]


In [316]:
pd.DataFrame(dist_to_ga['Florida'].values()).describe()

,0
count,142.000000
mean,337.961919
std,144.543301
min,1.335761
25%,388.724784
50%,415.330678
75%,421.862131
max,427.139693


In [317]:
pd.DataFrame(dist_to_ga['Alabama'].values()).describe()

,0
count,12.000000
mean,75.408657
std,46.230323
min,41.364905
25%,49.562724
50%,50.640955
75%,84.173838
max,161.716842


In [318]:
pd.DataFrame(dist_to_ga['Tennessee'].values()).describe()

,0
count,25.000000
mean,9.644941
std,2.841635
min,2.057784
25%,8.270898
50%,9.257004
75%,11.680576
max,14.731425


In [319]:
pd.DataFrame(dist_to_ga['South Carolina'].values()).describe()

,0
count,105.000000
mean,55.188245
std,20.092929
min,2.356079
25%,39.960461
50%,56.503895
75%,71.232295
max,106.960527


In [320]:
pd.DataFrame(dist_to_ga['North Carolina'].values()).describe()

,0
count,60.000000
mean,73.560841
std,23.057736
min,2.570491
25%,76.592661
50%,81.451586
75%,85.208131
max,88.496757


# Incorporate watershed boundaries

In [240]:
WBD_polygons = [shape(feature['geometry']) for feature in WBD_gj['features']]

In [271]:
poly = WBD_polygons[0]

In [273]:
for i, poly in enumerate(WBD_polygons):
    if not poly.is_valid:
        print(i)

In [277]:
for i, poly in enumerate(WBD_polygons):
    if poly.is_empty:
        print(i)

The polys generally do not appear to be closed. Could this be why they don't all show up correctly in the plot, and fail to fill the whole state's area?

Are we missing WBD polys that will prevent some record locations from being properly associated with a WBD?

In [278]:
poly.is_closed

False

In [345]:
ll = df[['lat', 'long']]

In [417]:
# TODO: create and use bounding boxes to get rough likelihood of containment before computing on the whole polygon
point_lookup_cache = {}
wbds = []
for lat, long in tqdm(ll.values):
    wbd_name = None
    try:
        wbd_name = point_lookup_cache[(long, lat)]
    except KeyError:
        point = Point(long, lat)
        for feat_idx, poly in enumerate(WBD_polygons):
            if poly.contains(point):
                wbd_name = WBD_gj['features'][feat_idx]['properties']['name']
                break
        point_lookup_cache[(long, lat)] = wbd_name
    wbds.append(wbd_name)

100%|██████████| 63797/63797 [00:08<00:00, 7219.17it/s] 


In [418]:
wbds.count(None)

4237

In [419]:
len(wbds) - wbds.count(None)

59560

In [420]:
set([name for name in wbds if name is not None])

{'030702040405',
 'Acorn Creek-Chattahoochee River',
 'Alapaha-Alapaha River',
 'Albany-Flint River',
 'Allatoona Creek-Lake Allatoona',
 'Allen Creek',
 'Altamaha Sound-Frontal Atlantic Ocean',
 'Amys Creek-Chattahoochee River',
 'Anneewakee Creek',
 'Arabia Swamp',
 'Arkaqua Creek-Nottely River',
 'Augusta Canal-Savannah River',
 'Avery Creek-Mill Creek',
 'Baileys Branch-Satilla River',
 'Baker Branch-Ogeechee River',
 'Bald Ridge Creek',
 'Baldwin Creek-Bear Creek',
 'Barbar Branch',
 'Barbers Creek-Satilla River',
 'Battle Creek-Ohoopee River',
 'Bear Branch',
 'Bear Creek',
 'Beaver Creek-Ohoopee River',
 'Beaver Ruin Creek',
 'Beaverdam Creek-Alcovy River',
 'Beaverdam Ditch-Savannah River',
 'Bell Creek-Potato Creek',
 'Ben Creek-Alapaha River',
 'Big Branch-Canoochee River',
 'Big Branch-Seventeen Mile River',
 'Big Cedar Creek',
 'Big Creek',
 'Black Mill Creek-Etowah River',
 'Blackbank River-Frontal Atlantic Ocean',
 'Blue Creek-Yellowjacket Creek',
 'Blue John Creek',
 'Bl

## Add watershed domain as a new column

In [421]:
df = df.assign(wbd=wbds)

Do 'None' WBD values correspond only to measurements outside of GA?

Do some correspond to the apparently non-covered areas in the GA map?

In [423]:
no_wbd_df = df[df['wbd'].isnull()]

In [424]:
len(no_wbd_df)

4237

In [425]:
no_wbd_df['state'].value_counts()

Florida           1695
South Carolina    1196
North Carolina     794
Tennessee          358
Alabama            144
Georgia             38
Name: state, dtype: int64

In [426]:
no_wbd_ga_df = no_wbd_df[no_wbd_df['state'] =='Georgia']

In [428]:
no_wbd_ga_df['SiteName'].value_counts()

Tybee Island Beach    21
North Beach           17
Name: SiteName, dtype: int64

Result:
 - This validates that the only issue with WBDs was graphing-related and that in fact all GA locations apart from Tybee Island Beach and North Beach match a WBD

Conclusion:
 - We should remove all records not corresponding to a GA watershed whose state field is not GA
 - Note that this protects the two Beach locations with no WBD, and retains those non-GA locations that correspond to in-GA watersheds.

In [429]:
non_ga_wbd_df = no_wbd_df[no_wbd_df['state'] != 'Georgia']

In [430]:
len(non_ga_wbd_df)

4199

In [431]:
df.drop(index=non_ga_wbd_df.index, inplace=True)

In [432]:
# check result
assert len(df[df['wbd'].isnull()]) == 38

In [86]:
def make_plot(fips, gdf):
    #fig = go.Figure(go.Scattergeo())
    fig = ff.create_choropleth(
        fips=fips,
        values=np.zeros(len(fips)),
        scope=['Georgia'],
        show_state_data=True,
        #colorscale="Reds", #colorscale,
        #binning_endpoints=endpts,
        round_legend_values=True,
        plot_bgcolor='rgb(229,229,229)',
        paper_bgcolor='rgb(229,229,229)',
        #legend_title=title,
        showlegend = False,
        county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
        exponent_format=False,
    )
    fig.layout.template = None
    fig.add_scattergeo(
        lat = gdf.geometry.x,
        lon = gdf.geometry.y,
        hoverinfo='none',
        #mode = 'markers', # 'markers+text'
        #text = texts,
        marker_size = 2,
        marker_color = 'rgb(235, 0, 100)'
    )
    hover_ix, hover = [(ix, t) for ix, t in enumerate(fig['data']) if t.text][0]
    if len(hover['text']) != len(GA_map):
        # hack fixes to hovertext while waiting on Issue 1429 to be fixed
        # https://github.com/plotly/plotly.py/issues/1429#issuecomment-506925578
        ht = pd.Series(hover['text'])

        no_dupe_ix = ht.index[~ht.duplicated()]

        hover_x_deduped = np.array(hover['x'])[no_dupe_ix]
        hover_y_deduped = np.array(hover['y'])[no_dupe_ix]

        new_hover_x = [x if type(x) == float else x[0] for x in hover_x_deduped]
        new_hover_y = [y if type(y) == float else y[0] for y in hover_y_deduped]

        fig['data'][hover_ix]['text'] = ht.drop_duplicates()
        fig['data'][hover_ix]['x'] = new_hover_x
        fig['data'][hover_ix]['y'] = new_hover_y
    fig.show()

In [ ]:
# doesn't properly scope to GA, shows whole world
# at least it's clear where the data is restricted to!
make_plot(fips, gdf)

In [ ]:
fig = ff.create_choropleth(
        fips=fips, values=values, scope=['Georgia'], show_state_data=True,
        #colorscale="Reds", #colorscale,
        binning_endpoints=endpts, round_legend_values=True,
        plot_bgcolor='rgb(229,229,229)',
        paper_bgcolor='rgb(229,229,229)',
        legend_title=title,
        county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
        exponent_format=False,
    )
fig.layout.template = None

## Lookup GIS place names associated with the recordings

Assumes that all NULL locations are already removed from the main dataframe

In [40]:
loc_lookup = fetch_geo_locs()

In [41]:
loc_lookup[3]['State']['name']

'Florida'

In [42]:
loc_lookup[155]['State']['name']

'Georgia'

Example location info

In [43]:
loc_lookup[0]

{'Block': {'FIPS': None, 'bbox': None},
 'County': {'FIPS': None, 'name': None},
 'State': {'FIPS': None, 'code': None, 'name': None},
 'status': 'OK',
 'executionTime': '0'}

In [44]:
loc_lookup[-1]

{'messages': ["FCC0001: The coordinate lies on the boundary of mulitple blocks, the block contains the clicked location is selected. For a complete list use showall=true to display 'intersection' element in the Block"],
 'Block': {'FIPS': '370399301002000',
  'bbox': [-83.7769, 35.212586, -83.702902, 35.25766]},
 'County': {'FIPS': '37039', 'name': 'Cherokee'},
 'State': {'FIPS': '37', 'code': 'NC', 'name': 'North Carolina'},
 'status': 'OK',
 'executionTime': '0'}

In [267]:
distance.distance(loc_pairs[3], loc_pairs[155]).miles

426.12898114111164

In [318]:
df.reset_index(inplace=True, drop=True)

In [329]:
row_loc_idxs = []
loc_pairs_for_df = []
seen_locs = {} # cache given high number of dupes

# for any row in full dataset, identify which unique location result
for idx in tqdm(range(len(df))):
    siteloc = df.iloc[idx]['SiteLocation']
    # lat, lon pair
    loc = tuple([float(x) for x in siteloc.split()])
    try:
        loc_idx = seen_locs[siteloc]
    except KeyError:
        # guaranteed to be unique by virtue of loc_pairs being created as unique values
        found_idxs = np.where(loc_pairs==loc)[0]
        if len(found_idxs) == 2 and len(set(found_idxs)) == 1:
            loc_idx = found_idxs[0]
        else:
            # maybe one coord matched but can't be both, so all idxs will have count of max 1 except for one
            loc_idx, count = Counter(found_idxs).most_common(1)[0]
            assert count == 2
        seen_locs[siteloc] = loc_idx
    row_loc_idxs.append(loc_idx)
    loc_pairs_for_df.append(loc)

100%|██████████| 63797/63797 [00:14<00:00, 4500.64it/s]


In [313]:
states_for_df = []
counties_for_df = []
for i in row_loc_idxs:
    states_for_df.append(loc_lookup[i]['State']['name'])
    counties_for_df.append(loc_lookup[i]['County']['name'])

In [375]:
df.insert(3, 'long', np.array(loc_pairs_for_df).T[1])
df.insert(3, 'lat', np.array(loc_pairs_for_df).T[0])
df.insert(3, 'county', counties_for_df)
df.insert(3, 'state', states_for_df)

In [476]:
df[['lat', 'long', 'SiteLocation', 'SiteName', 'wbd']].tail()

,lat,long,SiteLocation,SiteName,wbd
63792,32.0067,-84.2266,32.0067 -84.2266,Muckalee Creek at McLittle Bridge Rd,Bear Branch
63793,31.2387,-82.3231,31.2387 -82.3231,"U.S. Highway 84, Waycross",Caney Branch-Satilla River
63794,31.2933,-81.9571,31.2933 -81.9571,"U.S. 301, Nahunta",Raulerson Swamp-Satilla River
63795,31.2196,-81.8677,31.2196 -81.8677,"U.S. Highway 82, Nahunta",Barbers Creek-Satilla River
63796,31.2400,-81.8634,31.24 -81.8634,Warners Landing,Barbers Creek-Satilla River


We'll keep "SiteLocation" to allow easy checking of unique lat-long pairs

# Checkpoint for stage 2 of cleaning (geo)

In [434]:
df.to_csv("export_dataframe_stage2.csv", index=False)